<a href="https://colab.research.google.com/github/codingbunny111/codingbunny111/blob/main/dancer1_32.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## StyleGAN2 with adaptive discriminator augmentation (ADA)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Follow NVDIEA System requirement from https://github.com/NVlabs/stylegan2-ada


In [4]:
# Clone the StyleGAN2-ADA repository and go inside the directory

!git clone https://github.com/NVlabs/stylegan

fatal: destination path 'stylegan' already exists and is not an empty directory.


In [5]:
cd stylegan/


/content/stylegan


# Preparing Custom dataset

1- Download images from internet and put them in one folder.

2- All the images are square and the same size.

3- Code needs the dataset to be in .tfrecords format. We first need to convert our dataset to this format. 
   StyleGAN2-ADA has made a script that makes this conversion easy.

# Resizing the images

In [6]:

import cv2
import os

path = '/content/drive/MyDrive/hundred'


im_size = 32   # Input image resolution must be a power-of-two otherwise you will get error. 
                 #Pixel resolutions that are powers of 2 (512 x 512, 1024 x 1024, 2048 x 2048, etc).

images = []

for file in os.listdir(path):
        img = cv2.imread(path + '/' + file)  # reading that image as array
        img = cv2.resize(img, (im_size, im_size))
        print(img.shape)
        images.append(img)
        # Save the image in Output Folder
        cv2.imwrite('/content/stylegan/datasets/resize/' + str(file) + '_resized.jpg', img)


(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32,

# Converting images into .tfrecords

In [9]:
!pip install tensorflow-gpu==1.15.0

     |████████████████████████████████| 411.5 MB 7.3 kB/s 
     |████████████████████████████████| 50 kB 7.4 MB/s 
     |████████████████████████████████| 3.8 MB 56.0 MB/s 
     |████████████████████████████████| 503 kB 69.6 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=3b0855633f73ca3dfb31548e490cd198085fcf3b388713c3ad7c05d875c2eefc
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.7.0
    Uninstalling tensorflow-estimator-2.7.0:
      Successfully uninstalled tensorflow-estimator-2.7.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.7.0
    Uninstalling tensorboard-2.7.0:
      Successfully uninstalled tensorboard-2.7.0
  Attempting uninstall: gast
    Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Succe

In [10]:
# Below command will create a multi-resolution .tfrecord file in /custom_dataset/tfrecords_dataset/ folder.
!pip install tqdm


!python dataset_tool.py create_from_images /content/stylegan/datasets/tf_data/ /content/stylegan/datasets/resize




Loading images from "/content/stylegan/datasets/resize"
Creating dataset "/content/stylegan/datasets/tf_data/"



dataset_tool.py:87: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  'data': tf.train.Feature(bytes_list=tf.train.BytesList(value=[quant.tostring()]))}))
Added 160 images.


In [ ]:
#Training StyleGAN2-ADA

# snap is how often you want to save the model and sample results
# res is what image resolution you want to train on
# augpipe is augmentation pipes, such as 'blit', 'geom', 'color', 'filter', 'noise', 'cutout' or combination of these



!python train.py --outdir ./results --snap=5 --data=/content/stylegan/datasets/tf_data/ --augpipe=bgcfnc --res=32




Creating the run dir root: results
Creating the run dir: results/00000-sgan-tf_data-1gpu
Copying files to the run dir
dnnlib: Running training.training_loop.training_loop() on localhost...



Streaming data using training.dataset.TFRecordDataset...


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`





Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Instructions for updating:
Use `tf.compat.v1.data.get_output_types(dataset)`.
Instructions for updating:
Use `tf.compat.v1.data.get_output_shapes(dataset)`.
Instructions for updating:
Use `tf.compat.v1.data.get_output_types(iterator)`.
Instructions for updating:
Use `tf.compat.v1.data.get_output_shapes(iterator)`.
Instructions for updating:
Use `tf.compat.v1.data.get_output_classes(iterator)`.
Dataset shape = [3, 32, 32]
Dynamic range = [0, 255]
Label size    = 0
Constructing networks...







G                           Params    OutputShape       Weight

In [ ]:
# There are many other arguments that you can modify, feel free to check the train.py code to learn more about the arguments.
# Once you run the command, it will start training and periodically save the result and the model file (.pkl) based on the snap arguments
# that you provided (In this case, every 10kimg). Once you think that the result is good enough or the FID starts to plateau, you can stop training and use the last saved .pkl file.

# Once you have the model file you can generate images using this command.

#!python generate.py --outdir=out --trunc=0.5 --seeds=600-605 --network={path_to_pkl_model_file}

## As We don't have that much memory which can train styleGAN2. So we will download any pretrained model on styleGAN2 and see how to generate images from that

https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada/pretrained/

In [ ]:
# Generate class conditional CIFAR-10 images (Fig.17 left, Car)
#!python generate.py --outdir=out --trunc=1 --seeds=0-35 --class=1 --network=https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada/pretrained/cifar10.pkl

!python generate.py --outdir=out1 --trunc=1 --seeds=0-10 --class=1 --network=https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada/pretrained/cifar10.pkl

Loading networks from "https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada/pretrained/cifar10.pkl"...
Setting up TensorFlow plugin "fused_bias_act.cu": Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Loading... Done.
Generating image for seed 0 (0/11) ...
Generating image for seed 1 (1/11) ...
Generating image for seed 2 (2/11) ...
Generating image for seed 3 (3/11) ...
Generating image for seed 4 (4/11) ...
Generating image for seed 5 (5/11) ...
Generating image for seed 6 (6/11) ...
Generating image for seed 7 (7/11) ...
Generating image for seed 8 (8/11) ...
Generating image for seed 9 (9/11) ...
Generating image for seed 10 (10/11) ...


2021-08-02 16:52:11.743285: W tensorflow/core/common_runtime/bfc_allocator.cc:237] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.14GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2021-08-02 16:52:11.743326: W tensorflow/core/common_runtime/bfc_allocator.cc:237] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.14GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
C:\Users\Lenovo\anaconda3\envs\myenv\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Lenovo\anaconda3\envs\myenv\lib\site-packages\tensorflow\python\framework\d